# Automatic scoring of x-ray images

## 1. Preprocessing

Import dependencies

In [14]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# PIL to open & manipulate images
from PIL import Image, ImageOps, ImageChops

# for messages in loops
from IPython.display import clear_output

# for train/test split
from sklearn.model_selection import train_test_split

# for one-hot encoding
from sklearn.preprocessing import LabelBinarizer

# keras
from tensorflow.contrib.keras.python.keras import backend as K
from tensorflow.contrib.keras.python.keras.models import Sequential
from tensorflow.contrib.keras.python.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.contrib.keras.python.keras import callbacks


Define image format & random seeds

In [2]:
# image format -> (rows, cols, channels)
K.set_image_data_format("channels_last")
# fix random seed for reproducibility
seed = 1
np.random.seed(seed)
tf.set_random_seed(seed)

Import csv

In [ ]:
df = pd.read_csv(filepath_or_buffer="/data/deepxray/data/rau_data/merged.csv")
#df = pd.read_csv(filepath_or_buffer="/Volumes/deepxray/data/rau_data/merged.csv")
df = df.loc[df['body_part'].isin(["HAND_LEFT", "HAND_BOTH"])]
names = df["sop_iuid"].as_matrix()
df.head()

Load images and labels into numpy arrays

In [ ]:
root = "/data"
#root = "/Volumes"
dirname = "deepxray/joint_detection/output"

# function to remove black bars from image (preparation for histogram equalization)
def trim(im):
    # create a black image with the same size as original
    bg = Image.new(mode=im.mode, size=im.size, color=0)
    # get difference between image and background
    diff = ImageChops.difference(image1=im, image2=bg)
    # ensures, that the whole border gets removed (fuzzy border due to compression of jpg)
    diff = ImageChops.add(image1=diff, image2=diff, scale=2.0, offset=-100)
    bbox = diff.getbbox()
    if bbox:
        return im.crop(bbox)

# function to load and preprocess the image
def preprocess_img(xray_name, joint_name):
    # folder name
    folder_name = "predictions_extract_" + joint_name
    # file name
    file_name = xray_name + "_" + joint_name + ".jpg"
    
    # read image
    image = Image.open(fp=os.path.join(root, dirname, folder_name, file_name))
    image_w, image_h = image.size
    
    # histogram normalization (remaps the image -> lightest pixel = 255, darkest pixel = 0)
    img = trim(image)
    img_w, img_h = img.size
    img = ImageOps.autocontrast(image=img)
    offset = (int((image_w - img_w) / 2), int((image_h - img_h) / 2))
    image.paste(img, offset)
    
    # turn into np array
    data = np.asarray(a=image, dtype="uint8")
    return(data)


In [ ]:
def get_score(joint_number, index):
    score = df[joint_number].as_matrix()[index]
    return(score)

In [ ]:
joint_numbers = ["hand_left_1", "hand_left_2", "hand_left_3", "hand_left_4", "hand_left_5", 
                 "hand_left_6", "hand_left_7", "hand_left_8", "hand_left_9", "hand_left_10"]
joint_names = ["pip5", "mcp5", "pip4", "mcp4", "pip3", "mcp3", "mcp2", "pip2", "mcp1", "pip1"]


for i in range(10):
    print(joint_numbers[i] + " -> " + joint_names[i])



In [ ]:
images = []
labels = []
n_img = names.shape[0]

for i, xray_name in enumerate(names):
    if (i % 100) == 0:
        clear_output()
        print("Importing images, {0:.2f} % finished".format(i/n_img*100))
        
    for joint in range(10):
        try:
            img = preprocess_img(xray_name, joint_names[joint])
        except Exception: 
            pass
        else:
            lbl = get_score(joint_numbers[joint], i)
            if np.isnan(lbl):
                continue
            else:
                images.append(img)
                labels.append(lbl)

In [ ]:
images = np.array(images)
labels = np.array(labels)
print(images.shape)
print(labels.shape)
#print(labels)

In [ ]:
for i in range(10):    
    plt.imshow(images[i].squeeze(), cmap='gray')
    plt.show()

In [ ]:
images = np.array(images)
np.save(file="/data/joint_scoring/img_array.npy", arr=images)
#np.save(file="/Users/janickrohrbach/Desktop/joint_scoring/img_array.npy", arr=images)

In [ ]:
images = np.load(file="/data/body_part_classification/img_array.npy")
#images = np.load(file="/Users/janickrohrbach/Desktop/joint_scoring/img_array.npy")
images = images.reshape(images.shape[0], images.shape[1], images.shape[2], 1)

In [ ]:
# create four classes
labels[labels <= 25] = 0
labels[(labels > 25) & (labels <= 50)] = 1
labels[(labels > 50) & (labels <= 75)] = 2
labels[labels > 75] = 3

In [ ]:
img_train, img_test, labels_train, labels_test = train_test_split(images, labels,
                                                                  test_size=0.25,
                                                                  random_state=1,
                                                                  stratify=labels)

In [ ]:
np.save(file="/data/joint_scoring/img_train.npy", arr=img_train)
np.save(file="/data/joint_scoring/img_test.npy", arr=img_test)
np.save(file="/data/joint_scoring/labels_train.npy", arr=labels_train)
np.save(file="/data/joint_scoring/labels_test.npy", arr=labels_test)

#np.save(file="/Users/janickrohrbach/Desktop/joint_scoring/img_train.npy", arr=img_train)
#np.save(file="/Users/janickrohrbach/Desktop/joint_scoring/img_test.npy", arr=img_test)
#np.save(file="/Users/janickrohrbach/Desktop/joint_scoring/labels_train.npy", arr=labels_train)
#np.save(file="/Users/janickrohrbach/Desktop/joint_scoring/labels_test.npy", arr=labels_test)

In [3]:
img_train = np.load(file="/data/joint_scoring/img_train.npy")
img_test = np.load(file="/data/joint_scoring/img_test.npy")
labels_train = np.load(file="/data/joint_scoring/labels_train.npy")
labels_test = np.load(file="/data/joint_scoring/labels_test.npy")

#img_train = np.load(file="/Users/janickrohrbach/Desktop/joint_scoring/img_train.npy")
#img_test = np.load(file="/Users/janickrohrbach/Desktop/joint_scoring/img_test.npy")
#labels_train = np.load(file="/Users/janickrohrbach/Desktop/joint_scoring/labels_train.npy")
#labels_test = np.load(file="/Users/janickrohrbach/Desktop/joint_scoring/labels_test.npy")

In [4]:
# one hot encode outputs
labels_train = LabelBinarizer().fit_transform(labels_train)
labels_test = LabelBinarizer().fit_transform(labels_test)
num_classes = labels_test.shape[1]
print(num_classes)

4


## 2. Model

In [11]:
def conv_model():
    # create model
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(3, 3), padding="same", input_shape=(150, 150, 1),
                     activation="relu", kernel_initializer="he_normal"))
    model.add(Conv2D(filters=32, kernel_size=(3, 3), padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=32, kernel_size=(3, 3), padding="same", activation="relu"))
    model.add(Conv2D(filters=32, kernel_size=(3, 3), padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), padding="same", activation="relu"))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), padding="same", activation="relu"))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=128, kernel_size=(3, 3), padding="same", activation="relu"))
    model.add(Conv2D(filters=128, kernel_size=(3, 3), padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=128, kernel_size=(3, 3), padding="same", activation="relu"))
    model.add(Conv2D(filters=128, kernel_size=(3, 3), padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(units=256, kernel_initializer="he_normal",
                    activation="relu"))
    model.add(Dropout(rate=0.5))
    model.add(Dense(units=256, kernel_initializer="he_normal",
                    activation="relu"))
    model.add(Dropout(rate=0.5))
    model.add(Dense(units=num_classes, activation="softmax",
                    kernel_initializer="he_normal"))

    # Compile model
    model.compile(optimizer="adam", loss="categorical_crossentropy",
                  metrics=["accuracy"])

    return model

In [12]:
# build the model
model = conv_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 150, 150, 32)      320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 150, 150, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 75, 75, 32)        9248      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 75, 75, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 37, 37, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 37, 37, 64)        18496     
__________

In [15]:
# tensorboard callback
tb_callback = callbacks.TensorBoard(log_dir="/data/joint_scoring/tensorboard/2nd_model",
                                    histogram_freq=0, write_graph=True,
                                    write_images=True)

In [17]:
# Fit the model
model.fit(x=img_train, y=labels_train, batch_size=100, epochs=20, verbose=2,
          callbacks=[tb_callback], validation_data=(img_test, labels_test))

Train on 75413 samples, validate on 25138 samples
Epoch 1/20
100s - loss: 0.1368 - acc: 0.9569 - val_loss: 0.1433 - val_acc: 0.9565
Epoch 2/20
98s - loss: 0.1359 - acc: 0.9580 - val_loss: 0.1375 - val_acc: 0.9576
Epoch 3/20
98s - loss: 0.1316 - acc: 0.9582 - val_loss: 0.1432 - val_acc: 0.9582
Epoch 4/20
98s - loss: 0.1274 - acc: 0.9589 - val_loss: 0.1422 - val_acc: 0.9552
Epoch 5/20
98s - loss: 0.1269 - acc: 0.9587 - val_loss: 0.1548 - val_acc: 0.9591
Epoch 6/20
98s - loss: 0.1237 - acc: 0.9596 - val_loss: 0.1358 - val_acc: 0.9580
Epoch 7/20
98s - loss: 0.1191 - acc: 0.9606 - val_loss: 0.1439 - val_acc: 0.9560
Epoch 8/20
98s - loss: 0.1192 - acc: 0.9606 - val_loss: 0.1408 - val_acc: 0.9572
Epoch 9/20
97s - loss: 0.1155 - acc: 0.9610 - val_loss: 0.1446 - val_acc: 0.9556
Epoch 10/20
97s - loss: 0.1149 - acc: 0.9614 - val_loss: 0.1605 - val_acc: 0.9582
Epoch 11/20
97s - loss: 0.1109 - acc: 0.9623 - val_loss: 0.1600 - val_acc: 0.9564
Epoch 12/20
97s - loss: 0.1094 - acc: 0.9627 - val_loss:

In [18]:
# Final evaluation of the model
scores = model.evaluate(img_test, labels_test, verbose=0)
print("Error: %.2f%%" % (100 - scores[1] * 100))

Error: 4.19%


In [19]:
# Save the model
model.save('/data/joint_scoring/2nd_model.h5')